In [1]:
import pandas as pd
import os
import gensim
import numpy as np
import time
import pickle
import warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from gensim.models.fasttext import FastText as FT_gensim
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv1D, GlobalMaxPool1D, SimpleRNN
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

Using TensorFlow backend.


---
#### 권 대리님 모델 필요 
#### h5로 해서 나중에 수정해야함
---

In [2]:
from keras.models import model_from_json

# Reconfigure model in json file
with open("http_appid_model_20191215", "r") as fp:
      model = model_from_json(fp.read())

# Load weights to new model
model.load_weights("http_appid_weights_20191215.h5")

---
#### 권 대리님이 모델 만들때 쓰신 train 데이터 필요
---

In [3]:
http = pd.read_csv('http_classifier_train_20191212.csv')

---
#### 내가 ruleerror로 판단한 xdr 필요
---

In [4]:
test = pd.read_csv('modelinputhttp.csv')

---
#### 권 대리님이 label encoding한 객체 필요
----

In [9]:
import pickle

with open('http_id_encoder_20191216.p', 'rb') as f:
    encoding = pickle.load(f)

---
#### 권 대리님 모델에 넣기
----

In [5]:
from DNS_Encoding.data_nodns_del import data_nodns_del
from DNS_Encoding.make_fasttext_model_vector2 import make_fasttext_model_vector2
from DNS_Encoding.making_training_bydns import making_training_bydns
http_1 = data_nodns_del(http)
http_vector, http_modelinit, max_length_http = make_fasttext_model_vector2(http_1)

/data01/CSB/CSB_Jupyter/PROBE/DEMO/DNS_Encoding/data_nodns_del.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['request_host'][i] = data['dns_querey_name'][i]


In [6]:
from modelling.make_test_modelinput_65536_delH import make_test_modelinput_65536_delH

modelinput = pd.DataFrame(make_test_modelinput_65536_delH(http, test, http_modelinit, max_length_http)[0])

extra columns: {'age__10살이하'}
extra columns: {'gender__0'}


In [7]:
y_prob = model.predict(modelinput) 
y_classes = y_prob.argmax(axis=-1)

In [11]:
result = pd.DataFrame(columns = list(encoding.classes_), data= y_prob).transpose()

In [12]:
result.columns = [str(i) for i in range(len(result.columns))]

In [13]:
a = pd.DataFrame(columns = ['app_id', 'Prob'])
for i in range(len(result.columns)):
    a = pd.concat([a, pd.DataFrame({'app_id':result.iloc[:, i].sort_values(ascending = False)[:10].index, 'Prob':result.iloc[:, i].sort_values(ascending = False)[:10].values})])

In [14]:
a = a.reset_index(drop = True)

In [15]:
b = test[['summary_create_time', 'imsi']]

In [16]:
newdf = pd.DataFrame(np.repeat(b.values,10,axis=0))
newdf.columns = b.columns

,summary_create_time,imsi
0,1.57288e+09,T08xudGxfRjD7GKUsR1BnfDA==
1,1.57288e+09,T08xudGxfRjD7GKUsR1BnfDA==
2,1.57288e+09,T08xudGxfRjD7GKUsR1BnfDA==
3,1.57288e+09,T08xudGxfRjD7GKUsR1BnfDA==
4,1.57288e+09,T08xudGxfRjD7GKUsR1BnfDA==
...,...,...
92345,1.57296e+09,T0p38wciH4C8/GBV6pQjnXaA==
92346,1.57296e+09,T0p38wciH4C8/GBV6pQjnXaA==
92347,1.57296e+09,T0p38wciH4C8/GBV6pQjnXaA==
92348,1.57296e+09,T0p38wciH4C8/GBV6pQjnXaA==


In [17]:
newdf['rank'] = [1,2,3,4,5,6,7,8,9,10] * (int(len(newdf)/10))

In [18]:
output = pd.merge(newdf,a, left_index= True, right_index=True)

---
#### 확률값만 top10 뽑는데 유용한 코드
---

In [ ]:
A = np.array(model.predict_proba(modelinput))
N = 10
sorted_row_idx = np.argsort(A, axis=1)[:,A.shape[1]-N::]
col_idx = np.arange(A.shape[0])[:,None]
out = A[col_idx,sorted_row_idx]
